### Advent of Code 2018

Starting on 1 December 2018 and, every day until the 25 December 2018, a new two-part puzzle is available. Providing a correct answer to the first part of the puzzle will give you a gold star. If you complete both parts, you receive... two gold stars. The description for the second part of the puzzle is only shown after solving part one. 

I will not copy here the complete descriptions of the puzzle. Follow the links in each section to understand what is asked in each puzzle.

Let's start with some imports that are needed for the solutions 

In [5]:
import os
from collections import defaultdict, Counter
from itertools import cycle

### [Day 1](https://adventofcode.com/2018/day/1): Chronal Calibration

The input for the first puzzle is a list of integers, one per line, representing the frequency changes. Let's have a look at the first few lines in the input file.

In [39]:
with open('input/d1.txt', 'r') as f:
   input = f.read().strip().split()
        
input[:10]

['-8', '-18', '-14', '-10', '-7', '-10', '-16', '+19', '-5', '-7']

##### Part One
> Starting with a frequency of zero, what is the resulting frequency after all of the changes in frequency have been applied?

That's easy! The resulting frequency corresponds to the sum of all the frequency changes

In [42]:
def part1():
    chgs = [int(x) for x in input]
        
    return sum(chgs)

part1()

497

Submiting the output above will earn us our first star. On to part 2

##### Part Two
> What is the first frequency your device reaches twice?

Reading carefully the description we realize that we have to loop multiple times over the input frequency changes. After a quick search I found __[itertools.cycle](https://docs.python.org/3.6/library/itertools.html#itertools.cycle)__. This allow us to make an iterator that repeats indefinitely over the elementos of the input list. In each iteration, the resulting frequency is added to the set if not visited previously. The process stops when a match is found in the set.  

<b>Important:</b> Remember to add the initial frequency (zero) to the set. Otherwise the solution fails for the trivial case: [-1, +1]

In [43]:
def part2():
    chgs = [int(x) for x in input]
    
    f = 0                  #starting frequency
    visited = set([f])     #store the starting frequency
    for c in cycle(chgs): 
        f += c
        if f in visited:
            return f
        else:
            visited.add(f)

part2()

558

That's our second star. The first puzzle and first day are completed.

### [Day 2](https://adventofcode.com/2018/day/2): Inventory Management System

Today the input is a list of box IDs

In [50]:
with open('input/d2.txt', 'r') as f:
   ids = f.read().splitlines()
        
ids[:10]

['bazvmqthjtrnlosiecxyghkwud',
 'pazvmqbijirzlosiecxyghkwud',
 'pazvtqbmjtrnlosiecxyghkwzd',
 'pazvmqbfjtrjlosnlcxyghkwud',
 'pazvkqbfjtrtlosiecjyghkwud',
 'paztmqbfjtrnbosiecxyglkwud',
 'pazvmqbfjtunlosievxmghkwud',
 'pazvmqbfjtmngosiecyyghkwud',
 'jazvmqbfjtrnlosiecxygikpud',
 'pazvqqbfctrnlosimcxyghkwud']

##### Part One

Our objective is to identify the number of box IDs that contain a letter which appears exactly twice and those that have a letter that appears exactly three times. Multiplying these two numbers produces a "checksum" which is the answer to our puzzle.

> What is the checksumfor your list of box IDs?

In [47]:
def part1():   
    has_two = 0
    has_three = 0

    for i in ids:
        c = Counter(i)
        if len([k for k,v in c.items() if v == 2]) > 0:
            has_two += 1
        if len([k for k,v in c.items() if v == 3]) > 0:
            has_three += 1
            
    return has_two * has_three

part1()


8296

#### Part Two

The boxes full of prototype fabric will have IDs which differ by exactly one character at the same position in both strings

> What letters are common between the two correct box IDs?

Let's start by identifying the boxes that differ by exactly one letter at the same position.

In [55]:
def part2():    
    for boxID1 in ids:
        for boxID2 in ids:
            if len([i for i in range(len(boxID1)) if boxID1[i] != boxID2[i]]) == 1:
                print('Candidate boxes: {} and {}'.format(boxID1, boxID2))
                return (boxID1, boxID2)
            
(boxID1, boxID2) = part2()                

Candidate boxes: pazvmqbfjtrbeosiecxlghkwud and pazvmqbfotrbeosiecxlghkwud


We found our candidate boxes, we need just to find the letters that are common to both boxes - this is as simple as removing the differing character from either of the boxes.

In [58]:
ans = []
for i in range(len(boxID1)):
    if boxID1[i] == boxID2[i]:
        ans.append(boxID1[i])
        
"".join(ans)

'pazvmqbftrbeosiecxlghkwud'

That's our second star. The second puzzle and second day are completed.